In [5]:
%pip install seaborn
%pip install librosa
%pip install kagglehub
%pip install pandas
%pip install matplotlib

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense,
    Bidirectional, LSTM, Reshape, Permute, Multiply, Lambda, Activation
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow.keras.backend as K
import tensorflow as tf
#tf.debugging.set_log_device_placement(True) # TURNED ON FOR GPU STUFF YOU CAN TURN THIS OFF
from IPython.display import Audio
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
import kagglehub
import random

print("Is GPU available:", tf.test.is_gpu_available())
print("GPU devices:", tf.config.list_physical_devices('GPU'))
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging

tf.debugging.set_log_device_placement(False)
logging.getLogger('tensorflow').setLevel(logging.ERROR)

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Is GPU available: True
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
tess_path = kagglehub.dataset_download("ejlok1/toronto-emotional-speech-set-tess")

emotions = []
file_paths = []

for root, dirs, files in os.walk(tess_path):
    for file in files:
        if file.endswith('.wav'):
            emotion = os.path.basename(root) # take base name as emotion
            emotions.append(emotion)
            file_paths.append(os.path.join(root, file))

labels = [label.lower().split('_')[1] if '_' in label else label.lower() for label in emotions]

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
onehot_encoder = OneHotEncoder(sparse=False)
y_onehot = onehot_encoder.fit_transform(encoded_labels.reshape(-1, 1))
y_onehot = y_onehot.astype(np.float32)
y_onehot.shape



100%|██████████| 428M/428M [00:58<00:00, 7.65MB/s] 

Extracting model files...


(5600, 7)

In [8]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    file_paths, y_onehot, test_size=0.2, stratify=y_onehot, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42
)
display(len(X_train))
display(y_train.shape)

display(len(X_val))
display(y_val.shape)

display(len(X_test))
display(y_test.shape)

3584

(3584, 7)

896

(896, 7)

1120

(1120, 7)

In [9]:
def noise(data, noise_factor_range=(0.01, 0.05)):
    noise_factor = tf.random.uniform([], minval=noise_factor_range[0], maxval=noise_factor_range[1])
    noise = noise_factor * tf.random.normal(tf.shape(data))
    return data + noise

def random_gain(data, gain_range=(0.8, 1.2)):
    gain = tf.random.uniform([], minval=gain_range[0], maxval=gain_range[1])
    return data * gain

def augment_audio(data):
    # Add random noise
    data = tf.cond(tf.random.uniform([]) > 0.5, lambda: noise(data), lambda: data)
    # Adjust gain
    data = tf.cond(tf.random.uniform([]) > 0.5, lambda: random_gain(data), lambda: data)

    return data

In [21]:
def preprocess_audio(file_path, label, sr=16000, duration=2.5, n_mels=128):
    audio = tf.io.read_file(file_path)
    waveform, _ = tf.audio.decode_wav(audio, desired_channels=1)
    waveform = tf.squeeze(waveform)

    print(f"Processing file: {file_path}")
    print(f"Waveform shape: {tf.shape(waveform)}")

    # Calculate max_length as an integer
    max_length = int(sr * duration)

    # Trim or pad waveform to max_length
    waveform = tf.cond(
        tf.shape(waveform)[0] > max_length,
        lambda: waveform[:max_length],  # Truncate
        lambda: tf.pad(waveform, [[0, max_length - tf.shape(waveform)[0]]])  # Pad
    )

    # Compute STFT
    stft = tf.signal.stft(waveform, frame_length=400, frame_step=160, fft_length=512)
    spectrogram = tf.abs(stft)

    print(f"Spectrogram shape: {tf.shape(spectrogram)}")

    # Compute Mel-Spectrogram
    mel_filter = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=n_mels,
        num_spectrogram_bins=tf.shape(spectrogram)[-1],
        sample_rate=sr
    )
    mel_spectrogram = tf.tensordot(spectrogram, mel_filter, axes=1)

    print(f"Mel-Spectrogram shape before normalization: {tf.shape(mel_spectrogram)}")

    # Normalize the Mel-Spectrogram
    mel_spectrogram = tf.math.log(tf.clip_by_value(mel_spectrogram, 1e-5, tf.reduce_max(mel_spectrogram)))

    print(f"Mel-Spectrogram shape after normalization: {tf.shape(mel_spectrogram)}")

    # Resize to ensure consistent dimensions
    mel_spectrogram = tf.image.resize(mel_spectrogram, [128, 79])

    print(f"Mel-Spectrogram shape after resizing: {tf.shape(mel_spectrogram)}")

    # Add channel dimension
    mel_spectrogram = tf.expand_dims(mel_spectrogram, axis=-1)

    return mel_spectrogram, label

In [85]:
#from tensorflow.keras.mixed_precision import set_global_policy
#set_global_policy('mixed_float16')

In [22]:
def attention_mechanism(inputs):
    attention = Dense(1, activation='tanh')(inputs)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = Lambda(lambda x: K.expand_dims(x, axis=-1))(attention)
    weighted_inputs = Multiply()([inputs, attention])
    return weighted_inputs

In [23]:
input_shape = (128, 79, 1)
inputs = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Reshape for RNN
x = Reshape((x.shape[1], -1))(x)

x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Dropout(0.3)(x)

x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Dropout(0.3)(x)

x = attention_mechanism(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

num_classes = y_onehot.shape[1]
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 79, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_4 (Conv2D)              (None, 128, 79, 32)  320         ['input_3[0][0]']                
                                                                                                  
 max_pooling2d_4 (MaxPooling2D)  (None, 64, 39, 32)  0           ['conv2d_4[0][0]']               
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 64, 39, 32)  128         ['max_pooling2d_4[0][0]']  

In [24]:
def create_optimized_dataset(file_paths, labels, batch_size=32, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    
    # Shuffle dataset
    if shuffle:
        dataset = dataset.shuffle(len(file_paths))
    
    # Map preprocessing
    dataset = dataset.map(preprocess_audio, num_parallel_calls=tf.data.AUTOTUNE)

    # Batch and prefetch
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# Create datasets
train_dataset = create_optimized_dataset(X_train, y_train, batch_size=32)
val_dataset = create_optimized_dataset(X_val, y_val, batch_size=32, shuffle=False)

for batch_idx, (inputs, labels) in enumerate(train_dataset.take(4)):
    print(f"Batch {batch_idx + 1}:")
    print("Inputs (mel spectrograms):")
    print(inputs.numpy())  # Convert to NumPy for easier inspection
    print("Labels:")
    print(labels.numpy())
    print(f"Are inputs all zeros? {np.all(inputs.numpy() == 0)}")
    print(f"Are labels all zeros? {np.all(labels.numpy() == 0)}")
    print(tess_path)


Processing file: Tensor("args_0:0", shape=(), dtype=string)
Waveform shape: Tensor("Shape:0", shape=(None,), dtype=int32)
Spectrogram shape: Tensor("Shape_2:0", shape=(None,), dtype=int32)
Mel-Spectrogram shape before normalization: Tensor("Shape_4:0", shape=(None,), dtype=int32)
Mel-Spectrogram shape after normalization: Tensor("Shape_5:0", shape=(None,), dtype=int32)


ValueError: in user code:

    File "<ipython-input-21-ed1c9048b6b0>", line 41, in preprocess_audio  *
        mel_spectrogram = tf.image.resize(mel_spectrogram, [128, 79])

    ValueError: 'images' contains no shape.


In [14]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# try:
#     tf.profiler.experimental.stop()
# except Exception as e:
#     print(f"Profiler stop error: {e}")

# try:
#     tf.profiler.experimental.start('logdir')
# except tf.errors.AlreadyExistsError:
#     print("Profiler already running. Skipping start.")

# tf.config.set_soft_device_placement(True)
tf.config.set_soft_device_placement(True)
with tf.device('/GPU:0'):
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=30,
    )
# tf.profiler.experimental.stop()

Epoch 1/30
112/112 [==============================] - 13s 49ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/30
112/112 [==============================] - 4s 39ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/30
112/112 [==============================] - 4s 39ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/30
112/112 [==============================] - 4s 40ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/30
111/112 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
for batch in train_dataset.take(1):
    inputs, labels = batch
    print("Input shape from dataset:", inputs.shape)
    print("Label shape from dataset:", labels.shape)
    
print("Model input shape:", model.input_shape)  # Should be (None, 128, 79, 1)
print("Model output shape:", model.output_shape)  # Should be (None, 7)

for batch in train_dataset.take(1):
    inputs, labels = batch
    print("Input batch max:", tf.reduce_max(inputs).numpy())
    print("Input batch min:", tf.reduce_min(inputs).numpy())
    print("Label batch:", labels.numpy())

for batch in train_dataset.take(1):
    inputs, labels = batch
    outputs = model(inputs)
    print("Output shape:", outputs.shape)

from tensorflow.keras.losses import CategoricalCrossentropy

loss_fn = CategoricalCrossentropy()
for batch in train_dataset.take(1):
    inputs, labels = batch
    outputs = model(inputs)
    loss = loss_fn(labels, outputs)
    print("Loss:", loss.numpy())


In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")